<h1>Importing Libraries</h1>



In [41]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

<h1>Constants Needed</h1>
<h2> Frame Manipulation </h2>

In [42]:
# Actions that we try to detect
ACTIONS = {
            'Alphabets' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Ñ', 'NG', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
            'Numbers' : ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
            'Family Words': ['Kuya', 'Ate', 'Nanay', 'Tatay', 'Kapatid', 'Anak', 'Tiya', 'Tiyo', 'Lolo', 'Lola']
}
        
CATEGORIES = ['Alphabets', 'Numbers', 'Family Words']


# Thirty videos worth of data
NO_SEQUENCES = 30

# Videos are going to be 30 frames in length
SEQUENCE_LENGTH = 30

# Folder start
START_FOLDER = 30


<h2> Folder Paths </h2>

In [43]:
# Path for exported data, numpy arrays
NP_DATA_PATH = os.path.join('NP_DATA')

#Path for exported data, mp4 format
VID_CLEAR_DATA_PATH = os.path.join('VID_CLEAR_DATA')

#Path for exported data, mp4 format w/ landmarks
VID_MP_DATA_PATH = os.path.join('VID_MP_DATA')

#Path for exported data, png format
IMG_CLEAR_DATA_PATH = os.path.join('IMG_CLEAR_DATA')

#Path for exported data, png format w/ landmarks
IMG_MP_DATA_PATH = os.path.join('IMG_MP_DATA')


<h1> Folder Creation </h1>


In [44]:
def setupFolders(no_sequences, actions, categories, data_path):
    
    for category in categories:
        for action in actions[category]: 
            for sequence in range(1,no_sequences+1):
                try: 
                    os.makedirs(os.path.join(data_path, categories, action, str(sequence)))
                except:
                    pass

<text>Run this cell to create the folders, if the folders are already created it will pass</text>

In [45]:
# Path Storage for numpy landmarks, images, videos
setupFolders(NO_SEQUENCES, ACTIONS, CATEGORIES, NP_DATA_PATH)
setupFolders(NO_SEQUENCES, ACTIONS, CATEGORIES, VID_CLEAR_DATA_PATH)
setupFolders(NO_SEQUENCES, ACTIONS, CATEGORIES, IMG_CLEAR_DATA_PATH)
setupFolders(NO_SEQUENCES, ACTIONS, CATEGORIES, VID_MP_DATA_PATH)
setupFolders(NO_SEQUENCES, ACTIONS, CATEGORIES, IMG_MP_DATA_PATH)

<h1>Initializing MP Model</h1>

In [46]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

<h1>Detecting Holistic Parts</h1>
<text>This block of code detects the part of the body that is within the scope of the mediapipe model.</text>


In [47]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

<h1>Drawing Landmarks</h1>
<text>This block of code puts the landmarks inputted by the media pipe model.</text>


In [48]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

<h1>Styling Landmarks</h1>
<text>This block of code styles the landmarks inputted by the media pipe model.</text>


In [49]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

<h1>Saving Frames</h1>


In [50]:
def saveImage(path, image, action, sequence,frame_num):
    try:
        cv2.imwrite(os.path.join(path, f'{action}{sequence}-{frame_num}.png'), image)
    except:
        print(path)

<h1>Saving Video Clips</h1>


In [51]:
def saveVideo(path, action, sequence, width, height):
    size = (width, height)
    result = object
    try:
        result = cv2.VideoWriter(os.path.join(path, f'{action}{sequence}.mp4'), cv2.VideoWriter_fourcc(*'MP4V'), 10, size)
    except:
        pass
    return result

<h1>Extract Keypoints</h1>

In [52]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

<h1>Capture Real Time Landmarks</h1>
<text>Comment down the save functions to test.</text>


In [ ]:
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    # NEW LOOP
    # Loop through actions
    
    try:
        for category in CATEGORIES:
            for action in ACTIONS[category]:
                
                # Loop through sequences aka videos
                for sequence in range(NO_SEQUENCES):
                        
                    #Start Record, Comment this if you want test
                    # clearVideo = saveVideo(path=os.path.join(VID_CLEAR_DATA_PATH, category, action, str(sequence+1)), action=action, sequence=sequence+1, width=int(cap.get(3)), height=int(cap.get(4)))
                    # mpipeVideo = saveVideo(path=os.path.join(VID_MP_DATA_PATH, category, action, str(sequence+1)), action=action, sequence=sequence+1, width=int(cap.get(3)), height=int(cap.get(4)))
                    
                    # Loop through video length aka sequence length
                    for frame_num in range(SEQUENCE_LENGTH):

                        # Read feed
                        ret, frame = cap.read()

                        if not ret:
                            raise StopIteration

                        # Flipping frame horizontally
                        frame = cv2.flip(frame, 1)


                        # Make detections
                        image, results = mediapipe_detection(frame, holistic)

                        # Draw landmarks
                        draw_styled_landmarks(image, results)

                        #Saved MP Image, Comment this if you want test
                        # saveImage(path=os.path.join(IMG_MP_DATA_PATH, category, action, str(sequence+1)), image=image, action=action, sequence=sequence+1, frame_num=frame_num+1)

                        #Adding MP Image, Comment this if you want test
                        # mpipeVideo.write(image)

                        
                        # NEW Apply wait logic, Enter to capture every starting frame
                        if frame_num == 0: 
                            cv2.putText(image, 'Press Enter to Start COLLECTION', (80,200), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                            cv2.putText(image, f'Collecting frames for {category} : {action} Video Number {sequence + 1}', (15,12), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            # Show to screen
                            cv2.imshow('OpenCV Feed', image)
                            if cv2.waitKey(0) & 0xFF == 10:

                                pass
                        else: 
                            cv2.putText(image, f'Collecting frames for {category} : {action} Video Number {sequence + 1} | Frame Number {frame_num + 1}', (15,12), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                            # Show to screen
                            cv2.imshow('OpenCV Feed', image)

                            
                        
                        #Export Clear Images, Comment this if you want test
                        # saveImage(path=os.path.join(IMG_CLEAR_DATA_PATH, category, action, str(sequence+1)), image=frame, action=action, sequence=sequence+1, frame_num=frame_num+1)
                        
                        #Adding Clear Frame, Comment this if you want test
                        # clearVideo.write(frame)


                        #Export keypoints each frame, Comment this if you want test
                        # keypoints = extract_keypoints(results)
                        # npy_path = os.path.join(NP_DATA_PATH, category, action, str(sequence + 1), str(frame_num + 1))
                        # np.save(npy_path, keypoints)


                        #Raising an exception to fully break the loop
                        if cv2.waitKey(10) & 0xFF == ord('q'):

                            raise StopIteration
                    

                    #Stop the record of videos, Comment this if you want test
                    # clearVideo.release()
                    # mpipeVideo.release()


                    

                                      
    except StopIteration:
        pass

#Release all resources, Comment this if you want test
# clearVideo.release()
# mpipeVideo.release()


#Release window
cap.release()
cv2.destroyAllWindows()                   
    